### last update 18/06/2024
# __Tutorial on using the Gaia archive to access data on open clusters (with answers)__

## _Author: Orlagh Creevey_

## _This Hands-on has been prepared based on the Master Hands-on course given by Orlagh Creevey and Mathias Schultheis_

## _Some of the material from this notebook has been copied from a notebook prepared by Anthony Brown_

## __Before you start (if not already done)__
It is useful for you to be registered as a user of the Gaia archive.  It is not mandatory for this exercise, but if you plan on requesting data from the Gaia archive regularly this is a good idea. The registration is easy and quick (and free) and offers the following facilities:

* 20 GB of space to store query results
* the option to create and share your own tables ('user tables'),
* longer timeout for large or complex query jobs.

You can register [here](https://www.cosmos.esa.int/web/gaia-users/register).

## TOC:  <a class="anchor" id="toc"></a>
* [Introduction on using the Gaia archive](#intro-install)
    * [Login](#intro-login)
    * [List tables in archive](#intro-listtables)
    * [Simple query on a gaia table](#intro-goldensamplequery)
    * [Plot a HR diagram](#intro-goldhrdiag)
* [Gaia data on clusters](#cluster)
    * [Exercise 1: Corrected CMD considering their distance](#cluster-exercise1)
    * [Exercise 2: Proper motions of the stars](#cluster-exercise2)
    * [Exercise 3: Gaia query to select M67 members](#cluster-exercise3)
    * [Exercise 4: CMD of M67](#cluster-exercise4)
* [Parallax Bias](#parallax-bias)    
    * [Exercise 5: Correcting parallaxes for M67](#cluster-exercise5)
    * [Exercise 6: Different estimates of distance](#cluster-exercise6)    
    
        

## First install required Python packages  <a class="anchor" id="intro-install"></a>

We need the `astroquery` package to access the tools for programmatic access to the Gaia archive. For more information and examples see:
* [astroquery documentation](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)
* [Gaia archive python tutorial](https://www.cosmos.esa.int/web/gaia-users/archive/use-cases#ClusterAnalysisPythonTutorial)


In [ ]:
!pip install astroquery   # to use the gaia archive
#!pip install git+https://github.com/dinilbose/ezbasti   #to download basti isochrones automatically (information later)


## Python imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

from astroquery.gaia import Gaia
from astropy.table import Table

from itertools import chain

## Matplotlib settings

In [ ]:
linewidths = 2
axislinewidths = 2
lenticks = 6

#rc('font', family='sans-serif', size=18)
rc('xtick.major', size=lenticks)
rc('xtick.minor', size=lenticks * 2 / 3)
rc('ytick.major', size=lenticks)
rc('ytick.minor', size=lenticks * 2 / 3)
rc('lines', linewidth=linewidths)
rc('axes', linewidth=axislinewidths)

## __Login to the Gaia archive first__   (not necessary)  <a class="anchor" id="intro-login"></a>

For this exercise you must be logged in to the Gaia archive. The form in the next code cell asks for you archive user name. This is also needed further down.

*Your might be automatically logged out after a while. Just re-run this cell to login again.*

In [ ]:
#@title Provide your Gaia archive user name
username = 'ocreevey' #@param {type:"string"}

Gaia.login(user = username)


### Cleaning up to avoid archive errors

*You can skip this step if you have yet not logged in*

If you plan up uploading your tables to the archive, then you need to ensure that you do not have user tables with the same names as the ones you will be using here.  So you should delete them by doing the following query where 'bla' is the name of your user table `user_yourname.bla`

`Gaia.delete_user_table('bla')`



In [ ]:
#Gaia.delete_user_table('bla')

You can also delete the tables through the Gaia archive [web interface](https://gea.esac.esa.int/archive/) (after signing in). To delete a table in the archive, look the table up in your "User Tables" section in the menu to the left of the screen (see image below) and then check the table(s) to delete and click on the "delete" icon
(highlighted in the image).

<img src="https://catcopy.ipac.caltech.edu/ssw/hands-on/Gaia_login.png"></img>

## Find Gaia DR3 tables in the Gaia archive  <a class="anchor" id="intro-listtables"></a>
We start by generating an overview of the Gaia DR3 tables available from the [ESA Gaia archive](https://gea.esac.esa.int/archive/).

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    if 'gaiadr3' in table.get_qualified_name():
        print (table.get_qualified_name())

## A first simple query on GaiaDR3 to produce a HR diagram  <a class="anchor" id="intro-goldensamplequery"></a>

Request data for a subset of the sources in the `gaiadr3.goldsample_fgkm_stars` presented in the Golden Sample of Astrophysical Parameters and store the results in `gtable`.  We will select just the first 20 000 results in order to speed up the query.

__Note:__ There are six tables from this work that are published on the Gaia archive and are found under Performance Verification Papers.  and is found in the above list `gaiadr3.goldsample_fgkm_stars` in the Gaia archive.

Let's create a job in order to query the archive and put the results in the table `gold`.

Then print out details of the new table.


In [ ]:
job = Gaia.launch_job_async("SELECT top 20000 source_id, teff_gspphot, radius_flame, lum_flame, age_flame \
FROM  gaiadr3.gold_sample_fgkm_stars")

gold = job.get_results()

In [ ]:
print(gold.info)

The next code block makes a HR diagram from the data using effective temperature and luminosity.

In [ ]:

teff = gold['teff_gspphot']
lum= gold['lum_flame']

fig, axcmd = plt.subplots(1, 1, figsize=(7, 7))

axcmd.hexbin(teff, np.log10(lum), bins='log', mincnt=1)
axcmd.set_xlabel(r'$T_{\rm eff}$ [K]')
axcmd.set_ylabel(r'$\log_{10} Lum$ [L$_{\odot}$]')
axcmd.invert_xaxis()


If you prefer to work in pandas, you can convert the table to a pandas data frame.   Then you can plot the figures directly from the pandas data frame
See the example below.   For the rest of the answers, I will use the above example for plots.


## __Now we will query the Gaia archive for sources in the direction of M67 / NGC 2682__  <a class="anchor" id="cluster"></a>

The approximate coordinates in galactic latitude and longitude of M67 are (215.37699, + 31.37026).  One could also query in ra, dec by replacing (l,b) by (ra, dec) and using the ra, dec coordinates.
We launch a cone search around the coordinates and retrieve all sources that are within a certain distance from the center.

Once we recover the results, we will stock them in a file called m67_cone.   Then we will print the information related to the data that we recovered.


In [ ]:
job = Gaia.launch_job_async("SELECT *, DISTANCE(215.3769982238633, +31.37026, l, b) AS ang_sep \
FROM gaiadr3.gaia_source \
WHERE DISTANCE(215.3769982238633, +31.37026, l, b) < 0.5 \
AND phot_g_mean_mag < 18 \
AND parallax IS NOT NULL")

m67_cone = job.get_results()

In [ ]:
print(m67_cone.info)

### Exercise 1  <a class="anchor" id="cluster-exercise1"></a>

Now plot a CMD by using information from the parallaxes.

[Back to TOC](#toc)

In [ ]:
# Your plotting code here



As we have performed a cone search, we have recovered all sources in the direction of the cluster even if they are not part of the cluster.   We need to find a way to identify the cluster members.    
You can start by plotting different parameters such as the distribution of sources on the sky (ra, dec).  Does this allow you to distinguish the cluster members?


In [ ]:
# Your plotting code here



### Exercise 2   <a class="anchor" id="cluster-exercise2"></a>
Now plot the distribution of source in proper motion (pmra, pmdec).  Note that I will change the type of plotting diagram, but you can use whatever you prefer.

Members of clusters will move together through space, and therefore will have very similar proper motions.   You should try to identify a "clustering" of points in order to find the sources corresponding to M67.  

You can repeat the following plot several times, each time zooming in on the region of interest.  

The aim is to find the approximate central pmra and pmdec of M67, along with an estimate of the width of the "cluster".   Then we will use these numbers in the next query to retain only those sources.

[Back to TOC](#toc)

In [ ]:
# Your plotting code here


### Exercise 3   <a class="anchor" id="cluster-exercise3"></a>
Now repeat the query that we used earlier for a cone search but this time you should add some constraints on proper motion (pmra, pmdec).   You need to replace values XX, YY and ZZ in the following query by the central pmra, pmdec that you found above and an approximage 'width' of the propermotion space to search within.
Relaunch the query and save the output into a table `m67`

`job = Gaia.launch_job_async("SELECT *, DISTANCE(215.3769982238633, +31.37026, l, b) AS ang_sep \
FROM gaiadr3.gaia_source \
WHERE DISTANCE(215.3769982238633, +31.37026, l, b) < 0.5 \
AND phot_g_mean_mag < 18 \
AND parallax IS NOT NULL \
AND abs(pmra - XX) < ZZ \
AND abs(pmdec - YY) < ZZ")`



[Back to TOC](#toc)

In [1]:
## your query here


### Exercise 4   <a class="anchor" id="cluster-exercise4"></a>
Now plot the CMD of the cluster using (1) apparent G magnitude, and (2) absolute G magnitude.

What parts of the cluster can you identify?

Which diagram looks cleaner?  Why?

[Back to TOC](#toc)

In [ ]:
# your code here 


In [ ]:
# your code here

### Parallax Zeropoint Bias  <a class="anchor" id="parallax-bias"></a>
Let's apply the parallax zeropoint correction, see [Lindegren et al. 2020](https://ui.adsabs.harvard.edu/abs/2021A%26A...649A...4L/abstract).   
A python code was made available by the co-author Pau Ramos and it can be installed using pip.  See https://gitlab.com/icc-ub/public/gaiadr3_zeropoint

[Back to TOC](#toc)

In [ ]:
pip install gaiadr3-zeropoint

In [ ]:
from zero_point import zpt
zpt.load_tables()

In [ ]:
phot_g_mean_mag = m67['phot_g_mean_mag']
nu_eff_used_in_astrometry = m67['nu_eff_used_in_astrometry']
pseudocolour=m67['pseudocolour']
ecl_lat=m67['ecl_lat']
astrometric_params_solved=m67['astrometric_params_solved']

parcor=zpt.get_zpt(phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved)
parallax_cor = m67['parallax'] - parcor


### Exercise 5  <a class="anchor" id="cluster-exercise5"></a>

Now you can recalculate the absolute magnitudes of the individual cluster members and re-plot a HR diagram

[Back to TOC](#toc)

In [ ]:
abs_gmag_dist = m67['phot_g_mean_mag']+5*np.log10(parallax_cor)-10
fig, axhrd = plt.subplots(1, 1, figsize=(7, 7))
bp_rp = m67['phot_bp_mean_mag'] - m67['phot_rp_mean_mag']
axhrd.hexbin(bp_rp, abs_gmag_dist, bins='log', mincnt=1)
axhrd.set_xlabel(r'$(G_\mathrm{BP}-G_\mathrm{RP})$')
axhrd.set_ylabel(r'$M_G$')
axhrd.invert_yaxis()

plt.show()

### Exercise 6  <a class="anchor" id="cluster-exercise6"></a>

Because the cluster is at the same distance from the Sun (apart from a certain width), we can then assume one single value for the distance.  
Calculate the mean distance to the cluster.

[Back to TOC](#toc)

In [ ]:
wts = 1./(m67['parallax_error'])**2
meandistance = 1000./np.median(m67['parallax'])
meandistance_wtd = 1000./np.average(m67['parallax'],weights=wts)
meandistance_cor = 1000./np.mean(parallax_cor)
meandistance_cor_wtd = 1000./np.average(parallax_cor,weights=wts)
print(meandistance)
print(meandistance_wtd)
print(meandistance_cor)
print(meandistance_cor_wtd)

In [ ]:
absmag_dist = m67['phot_g_mean_mag'] + 5 - 5*np.log10(meandistance_cor_wtd)
fig, axhrd = plt.subplots(1, 1, figsize=(7, 7))

axhrd.hexbin(bp_rp, absmag_dist, bins='log', mincnt=1)
axhrd.set_xlabel(r'$(G_\mathrm{BP}-G_\mathrm{RP})$')
axhrd.set_ylabel(r'$M_{G,dist}$')
axhrd.invert_yaxis()

plt.show()


In [ ]:
df=m67.to_pandas()

In [ ]:
df["absmag_dist"] = df['phot_g_mean_mag'] + 5 - 5*np.log10(meandistance_cor_wtd)
df.to_csv('m67_magbprp.csv')

## __Follow up work__

1. Repeat the above exercise but now for the Pleiades cluster

# END OF EXERCISE

In [ ]:
## Let's write one of the tables to save.  It can be found on the left panel
## if you click the folder icon.  Then you can download it.

gold.write('gold.vot',format='votable',overwrite=True)


In [ ]:
## this is how you read in a vot table
data = Table.read('gold.vot')
